In [5]:
import functionlib as fl
import numpy as np
import pandas as pd

In [ ]:
def payoff_barrier_option(path, K, T, B, observ_freq, option_type):
    """
    Computes the payoff of a barrier option given a price path. 
    Type can be:
    - Knock-in Call ('kic') 
    - Knock-out Call ('koc')
    - Knock-in Put ('kip') 
    - Knock-out Put ('kop')

    Observation are made every observ_freq days.
    """

    time_steps = T*252
    observation_dates = np.arrange(observ_freq, time_steps+1, observ_freq)

    path_at_observation = [path.iloc[observation-1] for observation in observation_dates]
    
    max_price = max(path)
    min_price = min(path)

    if option_type=='kic':
        

        







    elif option_type=='koc':

    

    elif option_type=='kip':


    elif option_type=='kop':

In [23]:
T=1
n_observ = 63

time_steps = T*252
observation_dates = np.arange(n_observ, time_steps+1, n_observ)

path = fl.monte_carlo(100, 1, 0.02, 0.03, 1)

path_at_observation = [path.iloc[observation-1] for observation in observation_dates]

path_at_observation


[1    101.552761
 Name: 62, dtype: float64,
 1    101.661245
 Name: 125, dtype: float64,
 1    102.330609
 Name: 188, dtype: float64,
 1    101.334106
 Name: 251, dtype: float64]